In [1]:
import duckdb
import pandas as pd
import numpy as np

In [2]:
# # read csv file

# with duckdb.connect(database='data.db', read_only=False) as con:
#     con.execute("create or replace table wifi_sessions as select * from 'data/hackacity_wifi_data.csv'")

In [3]:
# with duckdb.connect(database='data.db', read_only=False) as con:
#     con.execute("""INSTALL spatial;
# LOAD spatial;""")
#     con.execute("create or replace table access_points as select * from st_read('data/aps_hackacity.xlsx', open_options = ['HEADERS=FORCE'])")

In [4]:
# with duckdb.connect(database='./data.db', read_only=False) as con:
#    access_points_data = con.execute("select * from access_points").fetch_df()

# access_points_data

In [5]:
# access_points_data = pd.read_excel('data/aps_hackacity.xlsx', na_values=np.nan)

In [6]:
with duckdb.connect(database='./data.db', read_only=False) as con:
   wifi_sessions_data = con.execute("select * from wifi_sessions").fetch_df()

wifi_sessions_data

wifi_sessions_data = wifi_sessions_data.drop(columns='column0')

In [7]:
SEED = 42

In [8]:
wifi_sessions_data['acctstarttime'] = wifi_sessions_data['acctstarttime'].astype('int64') // 10**9  # Divide by 10^9 to convert from nanoseconds to seconds

In [9]:
X = wifi_sessions_data.copy()

In [10]:
X.isna().sum()

acctsessionid               0
acctstarttime               0
upload                      0
download                    0
acctsessiontime       1861792
callingstationid            0
calledstationid             0
calledstation_ssid     275346
dtype: int64

In [11]:
X = X.apply(lambda col: col.fillna(col.mode()[0]))

In [12]:
X.isna().sum()

acctsessionid         0
acctstarttime         0
upload                0
download              0
acctsessiontime       0
callingstationid      0
calledstationid       0
calledstation_ssid    0
dtype: int64

In [13]:
CAT_COLS=['acctsessionid', 'callingstationid', 'calledstationid', 'calledstation_ssid']

In [14]:
wifi_sessions_data.head(2)

,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,fbfe95a31b975faa4b79968f95d834f2204ff050c4cb17...,1726668,0,0,0.0,412a9bf588f4a609cd8e4c8fc8b69482d6f69079f274de...,e0c8b4c72e48ff3726f36a987f29bab9b3d718646b3a6e...,eduroam
1,7330b1f18b414f72c4895700952fb9f2f5aa4cd3c15369...,1726664,0,0,0.0,d5f9cf3d6db078eec48f1f235072ddcec82917cf70aca7...,7b85b2daca64da5d39240d583c6097589a9a5dab4215b0...,eduroam


In [15]:
from process import CategoricalEncoder, PostProcess

#CAT_COLS=['Title', 'Type', 'Hotspot', ' MAC Radius', 'Zone', 'Street', 'Parish']

cat_encoder = CategoricalEncoder(categorical_columns=CAT_COLS)
cat_encoder.fit(X)
X_encoded = cat_encoder.transform(X)
X_encoded

,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,6543791,1726668,0,0,0.0,126221,244,1
1,2992450,1726664,0,0,0.0,413829,135,1
2,3715121,1726124,0,0,6.0,312763,206,0
3,1730167,1726102,2,30,32.0,199339,175,1
4,531229,1726106,0,1,6.0,278388,100,1
...,...,...,...,...,...,...,...,...
13983113,728134,1726660,0,0,0.0,395645,32,0
13983114,948633,1726102,0,0,0.0,384999,193,0
13983115,4821705,1726113,0,0,6.0,303431,227,0
13983116,2681722,1726117,3,6,6.0,488592,213,1


In [16]:
# smote.fit_resample(X=X_encoded, y=y)

In [17]:
from sklearn.mixture import GaussianMixture

# Train GMM
gmm = GaussianMixture(n_components=5, random_state=SEED)
gmm.fit(X_encoded)

GaussianMixture(n_components=5, random_state=42)

In [18]:
X_encoded.head()

,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,6543791,1726668,0,0,0.0,126221,244,1
1,2992450,1726664,0,0,0.0,413829,135,1
2,3715121,1726124,0,0,6.0,312763,206,0
3,1730167,1726102,2,30,32.0,199339,175,1
4,531229,1726106,0,1,6.0,278388,100,1


In [19]:
# Generate synthetic samples
synthetic_data = gmm.sample(n_samples=len(X_encoded))[0]
synthetic_data_df = pd.DataFrame(synthetic_data, columns=X_encoded.columns)
synthetic_data_df.head()

/Users/pedro.machado/Desktop/hackacity/hackacity/.venv/lib/python3.13/site-packages/sklearn/mixture/_base.py:436: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,2.920114e+05,1.723466e+06,-0.001914,0.000239,78605.572414,356303.270007,90.972815,0.219300
1,6.638380e+06,1.722386e+06,-0.000228,0.001465,60664.656666,164663.589567,113.416724,-0.283400
2,3.195533e+06,1.723297e+06,-0.000291,-0.000601,75872.004826,41254.074986,129.956633,0.592402
3,4.481019e+06,1.724391e+06,-0.001960,0.000206,93861.950526,511514.296894,233.634586,-0.203831
4,5.876371e+06,1.725855e+06,-0.000718,-0.001480,118381.953107,273674.869569,121.922056,0.307959


In [20]:
post_process = PostProcess(categorical_columns=CAT_COLS, original_data=X_encoded)
synthetic_data_df = post_process.transform(df   =synthetic_data_df)
synthetic_data_df

,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,292011,1723465,0,0,78605.572414,356303,91,0
1,6638380,1722386,0,0,60664.656666,164664,113,0
2,3195533,1723296,0,0,75872.004826,41254,130,1
3,4481019,1724390,0,0,93861.950526,495274,234,0
4,5876371,1725855,0,0,118381.953107,273675,122,0
...,...,...,...,...,...,...,...,...
13983113,2833224,1720200,1,17,12750.947092,143585,153,0
13983114,4361125,1724772,0,0,86537.292458,473799,57,0
13983115,2980853,1722452,1,14,107993.664951,153215,155,0
13983116,5422479,1723777,0,0,161.383797,225366,92,0


In [21]:
synthetic_data_df=cat_encoder.decode(synthetic_data_df)
synthetic_data_df

Decoding column acctsessionid
Decoding column callingstationid
Decoding column calledstationid
Decoding column calledstation_ssid


,acctsessionid,acctstarttime,upload,download,acctsessiontime,callingstationid,calledstationid,calledstation_ssid
0,0b3afeb8254e6fb6cf1cf4f4659676a84ab2338d5a2abc...,1723465,0,0,78605.572414,b819dde4479bdf70d5de5dc65df1ae30db8e53d8121632...,5646124e6a2972e4c8e49aefd934d3f7609adcf99fb297...,Porto. Free Wi-Fi
1,ffa1d2655f17a92f8e68fbd4b0902a6ad7890fc5a2324b...,1722386,0,0,60664.656666,55013cee8b55a905f90fbfe2f0782de2ae51655dbb69d1...,679871ac7b59c2a7d76be1a74e9a7e3fb224faa494b1fb...,Porto. Free Wi-Fi
2,7b09e0468c6bfe0236ace270b9b70c24fbd8841fbb83e9...,1723296,0,0,75872.004826,15344f8cf8b209a81c5a908910fc5dafab4b056c7052e3...,7501b9e9437bf71fc10c363fcbe3fc6c36c5f447f93d22...,eduroam
3,ac78e9fe7f3bcf53b8c1bdd5b32e7c29c1fbdac24bc65b...,1724390,0,0,93861.950526,ffffef056751777a6c25898f8032327cb0b008bac5605e...,d764f74ac1085e17b042a6b3eabb78e74170dbc4e10198...,Porto. Free Wi-Fi
4,e2427e7c91810fb1d9472446f2c26bbde72ac23975eff1...,1725855,0,0,118381.953107,8d76cfd7d06febd312c2c3aee38b2feac19a06ed7d02d1...,6f2ed2e71e167341cbbce4447d8a9065f8370294352f18...,Porto. Free Wi-Fi
...,...,...,...,...,...,...,...,...
13983113,6d11a49809ab294cb65ed65e9638bf943abfb6415da8be...,1720200,1,17,12750.947092,4a2bcf9389fe011455edc14501377fd2ea88aff86e2f6d...,955fb9836e6bb10f3683896b6f04b25f0034d63ce0e3bf...,Porto. Free Wi-Fi
13983114,a7dc2b72f09f3e188820bbceead11cdbd7db40ac206e6d...,1724772,0,0,86537.292458,f4db801833b0153cc6d3cfbb05cfc3934c8c2a1f3e4330...,3c3d76861294fc3cc91f24bb4371475d70d7c031098fb6...,Porto. Free Wi-Fi
13983115,72be8cb23a1df0b2cbf71eaa4cc109999843b45c8912c5...,1722452,1,14,107993.664951,4f17a175e41155218e256c329bfa6a7eeef8ec6f7c5263...,960efb7312c32289b2e97a26cc8bbef1b1c9b03aa6e93d...,Porto. Free Wi-Fi
13983116,d0cc2a94ab3b19784eba6170a2d2fb2f9177ecda551fde...,1723777,0,0,161.383797,746e2b3bd0f303456af841fc828dd9f566361c5155068b...,565393e2960918f8b61b611ad1f28db8c8d206b682c5f2...,Porto. Free Wi-Fi


In [22]:
synthetic_data_df.to_parquet(compression="zstd", path="./synthetic_data.parquet")

TODO:
Coordenadas podem não dar match ao hotspot 